In [49]:
line_to_run= '''python -m nbconvert --to html --no-input "Reporte_Uvas.ipynb" --log-level CRITICAL
#python -m nbconvert --to webpdf --no-input "Reporte_Uvas.ipynb" --TagRemovePreprocessor.enabled=True --PDFExporter.latex_command="['pdflatex', '{filename}', '--landscape']" '''
import warnings
warnings.filterwarnings('ignore')

<div style="text-align:center; margin-top:20px; margin-bottom:10px;">
    <img src="bflogo.png" alt="Blossom Fruit Logo" style="width:600px; max-width:90%;"/>
</div>

<h1 style="text-align:center; font-size:32px;">
    Blossom Fruit International United States Weekly Grape Imports Analysis
</h1>
<h2 style = "text-align:center; font-size:18px;">
    November 3 - November 9
</h2>

In [50]:
import pandas as pd
master=pd.read_csv(r"C:\Users\artuv\OneDrive\Bases_Chile_y_UvasPeru\Master.csv")
master.rename(columns={'Dep_Week': 'Departure Week'},inplace=True)
master['Export Country']=master['Year Country'].str.split('-').str[1]

In [51]:
usa = master[(master['Country']=='U.S.A.') & (master['Specie']=='TABLE GRAPE')]
yearmax = usa['Year'].max()
weekmax = usa[usa['Year']==yearmax]['Departure Week'].max()
for i, row in usa.iterrows():
    w = row['Departure Week']
    y = row['Year']

    if ((w >= 40) and (y >= 2025)):
        usa.at[i, 'Season'] = '2025-2026'
    elif ((w >= 40) and (y == 2024)) or ((w <= 39) and (y == 2025)):
        usa.at[i, 'Season'] = '2024-2025'
    elif ((w >= 40) and (y == 2023)) or ((w <= 39) and (y == 2024)):
        usa.at[i, 'Season'] = '2023-2024'
    else:
        usa.at[i, 'Season'] = '2022-2023'


In [52]:

season_week=[]
for i in usa['Departure Week']:
    if i<40:
        season_week.append(i+13)
    else:
        season_week.append(i-39)
usa['Season Week']=season_week
usa=usa[usa['Season'].isin(['2023-2024','2024-2025','2025-2026'])]
x=usa.groupby(['Season Week','Season','Market']).agg({'Boxes':'sum'}).reset_index()


In [53]:
current_week=6
if ((current_week>0)&(current_week<14)):
    adjustment= 39
elif ((current_week>13)&(current_week<40)):
    adjustment= -13

## Weekly Imports of Chilean and Peruvian Grapes per Coast (Boxes)


In [54]:

# Step 2: all unique seasons sorted
seasons = sorted(x['Season'].unique(),reverse=True)

# Step 3: build and merge pivots with totals in between
merged = None

for s in seasons:
    # pivot for this season
    temp = x[x['Season'] == s]
    pivot = temp.pivot_table(
        index='Season Week',
        columns='Market',
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)

    # add a total column for this season
    pivot['TOTAL'] = pivot.sum(axis=1)

    # add season level to all columns
    pivot.columns = pd.MultiIndex.from_product([[s], pivot.columns])

    # merge with previous seasons
    if merged is None:
        merged = pivot
    else:
        merged = merged.join(pivot, how='outer')

# Assume your pivot DataFrame is called merged
# and its index represents numeric Departure Weeks
# Ensure the index is numeric and sorted
merged.index = pd.to_numeric(merged.index, errors='coerce')
merged = merged.sort_index()
idx = merged.index

# 1️⃣ Current week only
current = merged.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

# 2️⃣ Previous 4 (including current)
prev_4 = merged.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

# 3️⃣ Previous 8 (including current)
prev_8 = merged.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8  Weeks Aggregate Total']

# 4️⃣ Season to Date (week 1 through current)
ytd = merged.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# Combine them in order
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

summary.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_properties(**{'text-align': 'center'})


<small><sup>*</sup>Week 1 is the calender week 40 every year. i.e. Week 40= Week 1, Week 41= Week 2 etc. 

*If the current week is less than 4, and or 8 the respective totals will reflect only the season totals, the aggregate are not inclusive of previous seasons.</small>

## Imports per Coast by Country of Origin (Boxes)

In [55]:
y= usa.groupby(by=['Season Week','Season','Market','Export Country']).agg({'Boxes':'sum'}).reset_index()

In [56]:

# 1️⃣ Build merged_y
seasons = sorted(y['Season'].unique(),reverse=True)
merged_y = None

for s in seasons:
    temp = y[y['Season'] == s]

    # Pivot by market and export country
    pivot = temp.pivot_table(
        index='Season Week',
        columns=['Market', 'Export Country'],
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)

    # --- Add TOTAL within each market (Chile + Peru) ---
    for market in pivot.columns.get_level_values(0).unique():
        # sum across CHILE + PERU (row-wise) for this market
        market_total = pivot[market].sum(axis=1)
        pivot[(market, 'TOTAL')] = market_total

    # --- Order columns within each market: CHILE, PERU, TOTAL ---
    new_order = []
    for market in sorted(pivot.columns.get_level_values(0).unique()):
        for country in ['CHILE', 'PERU', 'TOTAL']:
            col = (market, country)
            if col in pivot.columns:
                new_order.append(col)
    pivot = pivot[new_order]

    # --- Add season as top-level column ---
    pivot.columns = pd.MultiIndex.from_tuples([(s,) + col for col in pivot.columns])

    # --- Merge all seasons horizontally ---
    if merged_y is None:
        merged_y = pivot
    else:
        merged_y = pd.concat([merged_y, pivot], axis=1)

# 2️⃣ Sort and ensure numeric weeks
merged_y.index = pd.to_numeric(merged_y.index, errors='coerce')
merged_y = merged_y.sort_index().fillna(0).astype(int)

# 3️⃣ Build summary rows
idx = merged_y.index
current = merged_y.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

prev_4 = merged_y.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

prev_8 = merged_y.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8 Weeks Aggregate Total']

ytd = merged_y.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# 4️⃣ Combine and format
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

# 5️⃣ Nicely formatted output (with commas)
summary.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_properties(**{'text-align': 'center'})


<small><sup>*</sup>There is no column for Chile in the 2025-2026 Season because there are no recorded Grape imports from Chile so far this Season </small>

In [57]:
usa_simplified = usa.copy()
usa_simplified['Variety group'] = usa['Variety group'].apply(
    lambda x: 'RED' if 'RED' in x else ('WHITE' if 'WHITE' in x else 'OTHERS')
)

y= usa_simplified.groupby(['Season Week','Season','Market','Variety group']).agg({'Boxes':'sum'}).reset_index()

<div style="page-break-before:always"></div>

## Weekly Imports by Variety Group/Color (Boxes)

In [58]:
# 1️⃣ Build merged_y
seasons = sorted(y['Season'].unique(),reverse=True)
merged_y = None

for s in seasons:
    temp = y[y['Season'] == s]

    # Pivot by Market and Variety group
    pivot = temp.pivot_table(
        index='Season Week',
        columns=['Market', 'Variety group'],
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)

    # --- Add TOTAL within each market (sum across varieties) ---
    for market in pivot.columns.get_level_values(0).unique():
        market_total = pivot[market].sum(axis=1)
        pivot[(market, 'TOTAL')] = market_total

    # --- Order columns within each market: RED → WHITE → OTHERS → TOTAL ---
    desired_order = ['RED', 'WHITE', 'OTHERS', 'TOTAL']
    new_order = []
    for market in sorted(pivot.columns.get_level_values(0).unique()):
        varieties = [v for v in desired_order if (market, v) in pivot.columns]
        new_order.extend([(market, v) for v in varieties])
    pivot = pivot[new_order]

    # --- Add season as top-level column ---
    pivot.columns = pd.MultiIndex.from_tuples([(s,) + col for col in pivot.columns])

    # --- Merge all seasons horizontally ---
    if merged_y is None:
        merged_y = pivot
    else:
        merged_y = pd.concat([merged_y, pivot], axis=1)

# 2️⃣ Sort and ensure numeric weeks
merged_y.index = pd.to_numeric(merged_y.index, errors='coerce')
merged_y = merged_y.sort_index().fillna(0).astype(int)

# 3️⃣ Build summary rows
idx = merged_y.index
current = merged_y.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

prev_4 = merged_y.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

prev_8 = merged_y.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8 Weeks Aggregate Total']

ytd = merged_y.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# 4️⃣ Combine and format
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

# 5️⃣ Nicely formatted output (with centered alignment)
summary.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_properties(**{'text-align': 'center'})


In [59]:
white=usa[usa['Variety group'].str.contains('WHITE')]
white_g=white.groupby(['Season Week','Season','Market','Variety group']).agg({'Boxes':'sum'}).reset_index()

## Weekly White Grape Imports

In [60]:
# 1️⃣ Build merged_y
seasons = sorted(white_g['Season'].unique(),reverse=True)
merged_y = None

for s in seasons:
    temp = white_g[white_g['Season'] == s]

    # Pivot by Market (we'll collapse Variety group immediately)
    pivot = temp.pivot_table(
        index='Season Week',
        columns=['Market', 'Variety group'],
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)

    # --- Collapse Variety group so we only have Market totals ---
    pivot = pivot.groupby(level=0, axis=1).sum()

    # --- Add TOTAL for the whole season (sum of all markets) ---
    pivot['TOTAL'] = pivot.sum(axis=1)

    # --- Wrap columns in a MultiIndex with Season as top level ---
    pivot.columns = pd.MultiIndex.from_tuples([(s, c) for c in pivot.columns])

    # --- Merge all seasons horizontally ---
    if merged_y is None:
        merged_y = pivot
    else:
        merged_y = pd.concat([merged_y, pivot], axis=1)

# 2️⃣ Sort and ensure numeric weeks
merged_y.index = pd.to_numeric(merged_y.index, errors='coerce')
merged_y = merged_y.sort_index().fillna(0).astype(int)

# 3️⃣ Build summary rows
idx = merged_y.index
current = merged_y.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

prev_4 = merged_y.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

prev_8 = merged_y.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8 Weeks Aggregate Total']

ytd = merged_y.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# 4️⃣ Combine and format
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

# 5️⃣ Nicely formatted output
from IPython.display import display
display(
    summary.style.format("{:,}")
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ])
    .set_properties(**{'text-align': 'center'})
)

## Weekly Red Grape Imports

In [61]:
r=usa[usa['Variety group'].str.contains('RED')]
r=r.groupby(['Season Week','Season','Market','Variety group']).agg({'Boxes':'sum'}).reset_index()

In [62]:
# 1️⃣ Build merged_y
seasons = sorted(r['Season'].unique(),reverse=True)
merged_y = None

for s in seasons:
    temp = r[r['Season'] == s]

    # Pivot by Market and Variety group
    pivot = temp.pivot_table(
        index='Season Week',
        columns=['Market', 'Variety group'],
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)
    
    # --- Add TOTAL within each market (sum across varieties) ---
    for market in pivot.columns.get_level_values(0).unique():
        market_total = pivot[market].sum(axis=1)
        pivot[(market, 'TOTAL')] = market_total

    # --- Order columns within each market: RED SEEDLESS → RED SEEDED → TOTAL ---
    desired_order = ['RED SEEDLESS', 'RED SEEDED', 'TOTAL']
    new_order = []
    for market in sorted(pivot.columns.get_level_values(0).unique()):
        varieties = [v for v in desired_order if (market, v) in pivot.columns]
        new_order.extend([(market, v) for v in varieties])
    pivot = pivot[new_order]

    # --- Add season as top-level column ---
    pivot.columns = pd.MultiIndex.from_tuples([(s,) + col for col in pivot.columns])

    # --- Merge all seasons horizontally ---
    if merged_y is None:
        merged_y = pivot
    else:
        merged_y = pd.concat([merged_y, pivot], axis=1)

# 2️⃣ Sort and ensure numeric weeks
merged_y.index = pd.to_numeric(merged_y.index, errors='coerce')
merged_y = merged_y.sort_index().fillna(0).astype(int)

# 3️⃣ Build summary rows
idx = merged_y.index
current = merged_y.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

prev_4 = merged_y.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

prev_8 = merged_y.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8 Weeks Aggregate Total']

ytd = merged_y.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# 4️⃣ Combine and format
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

# 5️⃣ Nicely formatted output (with centered alignment)
summary.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_properties(**{'text-align': 'center'})

In [63]:
black=usa[usa['Variety group'].str.contains('BLACK')]
black=black.groupby(['Season Week','Season','Market','Variety group']).agg({'Boxes':'sum'}).reset_index()

<div style="page-break-before:always"></div>

## Weekly Black Grape Imports

In [64]:
# 1️⃣ Build merged_y
seasons = sorted(black['Season'].unique(),reverse=True)
merged_y = None

for s in seasons:
    temp = black[black['Season'] == s]

    # Pivot by Market and Variety group
    pivot = temp.pivot_table(
        index='Season Week',
        columns=['Market', 'Variety group'],
        values='Boxes',
        aggfunc='sum'
    ).fillna(0)

    # --- Collapse Variety group level so only Market remains ---
    pivot = pivot.groupby(level=0, axis=1).sum()

    # --- Add TOTAL across all markets for the season ---
    pivot['TOTAL'] = pivot.sum(axis=1)

    # --- Add season as top-level column ---
    pivot.columns = pd.MultiIndex.from_tuples([(s, col) for col in pivot.columns])

    # --- Merge all seasons horizontally ---
    if merged_y is None:
        merged_y = pivot
    else:
        merged_y = pd.concat([merged_y, pivot], axis=1)

# 2️⃣ Sort and ensure numeric week index
merged_y.index = pd.to_numeric(merged_y.index, errors='coerce')
merged_y = merged_y.sort_index().fillna(0).astype(int)

# 3️⃣ Build summary rows
idx = merged_y.index

current = merged_y.loc[idx == current_week].sum().to_frame().T
current.index = [f'Week {current_week + adjustment}']

prev_4 = merged_y.loc[(idx >= current_week - 3) & (idx <= current_week)].sum().to_frame().T
prev_4.index = ['Previous 4 Weeks Aggregate Total']

prev_8 = merged_y.loc[(idx >= current_week - 7) & (idx <= current_week)].sum().to_frame().T
prev_8.index = ['Previous 8 Weeks Aggregate Total']

ytd = merged_y.loc[(idx <= current_week)].sum().to_frame().T
ytd.index = ['Season Totals to Date']

# 4️⃣ Combine and format final summary
summary = pd.concat([current, prev_4, prev_8, ytd])
summary = summary.fillna(0).astype(int)

# 5️⃣ Nicely formatted display (centered)
from IPython.display import display
display(
    summary.style.format("{:,}")
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ])
    .set_properties(**{'text-align': 'center'})
)


© 2025 Blossom Fruit International, LLC — All Rights Reserved.